<a href="https://colab.research.google.com/github/SantirTheK/Lab2-Valencia-Restrepo/blob/main/Lab2_RestrepoSantiago_ValenciaMateo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Caracteres válidos para el cifrado
ABECEDARIO_VALIDO = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")

# Clase auxiliar para convertir letras en unario y aplicar módulo
class ProcesadorUnario:
    def __init__(self, alfabeto):
        self.alfabeto = alfabeto

    def a_unario(self, letra):
        return ['1'] * self.alfabeto.index(letra)

    def modulo_26(self, cadena):
        while len(cadena) >= 26:
            cadena = cadena[26:]
        return cadena

    def unario_a_letra(self, cadena):
        return self.alfabeto[len(cadena)]

# Máquina de Turing para cifrado
class MaquinaDeTuring:
    def __init__(self, texto, clave):
        self.texto_original = texto.upper().replace(" ", "")
        self.clave_expandida = self._extender_clave(clave.upper(), len(self.texto_original))

        self.cintas = {
            'entrada': list(self.texto_original),
            'clave': list(self.clave_expandida),
            'salida': ['_' for _ in self.texto_original],
            'temporal': []
        }

        self.punteros = {nombre: 0 for nombre in self.cintas}
        self.estado = 'inicio'
        self.terminado = False
        self.estados_formales = {
            'inicio': 'q0',
            'restante': 'q1',
            'convertir': 'q2',
            'desplazar': 'q3',
            'fin': 'qf'
        }

        self.procesador = ProcesadorUnario(ABECEDARIO_VALIDO)

    def _extender_clave(self, clave, longitud):
        return (clave * ((longitud // len(clave)) + 1))[:longitud]

    def _leer_letras(self):
        l1 = self.cintas['entrada'][self.punteros['entrada']]
        l2 = self.cintas['clave'][self.punteros['clave']]
        return l1, l2

    def _escribir_salida(self, letra):
        pos = self.punteros['salida']
        self.cintas['salida'][pos] = letra

    def _mover_punteros(self):
        for key in self.punteros:
            if key != 'temporal':
                self.punteros[key] += 1
        self.punteros['temporal'] = 0
        self.cintas['temporal'] = []

    def paso(self):
        if self.estado == 'inicio':
            if self.punteros['entrada'] >= len(self.cintas['entrada']):
                self.estado = 'fin'
                self.terminado = True
                return

            l1, l2 = self._leer_letras()
            print(f"Transición: estado={self.estados_formales[self.estado]}, Entrada={l1}, Clave={l2}")
            u1 = self.procesador.a_unario(l1)
            u2 = self.procesador.a_unario(l2)
            self.cintas['temporal'] = u1 + u2
            self.estado = 'restante'

        elif self.estado == 'restante':
            self.cintas['temporal'] = self.procesador.modulo_26(self.cintas['temporal'])
            self.estado = 'convertir'

        elif self.estado == 'convertir':
            letra_resultado = self.procesador.unario_a_letra(self.cintas['temporal'])
            print(f"Escribiendo en cinta salida: {letra_resultado} en posición {self.punteros['salida']}")
            self._escribir_salida(letra_resultado)
            self.estado = 'desplazar'

        elif self.estado == 'desplazar':
            self._mover_punteros()
            self.estado = 'inicio'

        elif self.estado == 'fin':
            self.terminado = True

    def ejecutar(self, mostrar=True):
        pasos = 0
        while not self.terminado:
            estado_q = self.estados_formales[self.estado]
            if mostrar:
                print(f"\nPaso {pasos}: Estado = {estado_q}")
                for nombre in ['entrada', 'clave', 'salida', 'temporal']:
                    contenido = ''.join(self.cintas[nombre]) if self.cintas[nombre] else '_'
                    pos = self.punteros[nombre]
                    print(f"Cinta {nombre.capitalize()}: {contenido}")
                    print("                " + " " * pos + "^")
            self.paso()
            pasos += 1

        if mostrar:
            print("\nEstado final alcanzado: qf (fin)")
        return ''.join(self.cintas['salida'])

# Entrada interactiva del usuario
if __name__ == "__main__":
    def obtener_entrada(mensaje):
        while True:
            entrada = input(mensaje).strip()
            if entrada.isalpha() and all(c in ABECEDARIO_VALIDO for c in entrada):
                return entrada
            print("Entrada inválida. Usa solo letras mayúsculas A-Z, sin Ñ ni símbolos.")

    texto_usuario = obtener_entrada("Ingrese el texto a cifrar (A-Z, sin Ñ): ")
    clave_usuario = obtener_entrada("Ingrese la clave (A-Z, sin Ñ): ")

    maquina = MaquinaDeTuring(texto_usuario, clave_usuario)
    resultado = maquina.ejecutar(mostrar=True)
    print("\nTexto cifrado final:", resultado)


In [ ]:
def cifrar_vigenere_tm(mensaje_claro, palabra_clave, mostrar=True):
    maquina = MaquinaDeTuring(mensaje_claro, palabra_clave)
    return maquina.ejecutar()

mensaje = "MENSAJEDEPRUEBA"
clave = "CLAVE"
resultado = cifrar_vigenere_tm(mensaje, clave, mostrar=True)
print("\nTexto cifrado final:", resultado)


### Definición de la maquina de turing

Nuestra Máquina de Turing esta definida formalmente con los siguientes componentes:

Q (Conjunto de estados): El conjunto finito de estados de la maquina es:

q0 (Estado inicio): Estado inicial y principal. Lee el carácter actual del texto claro y de la clave. Realiza la conversión a unario de ambos caracteres y los concatena en la cinta temporal. Pasa a q1. Si no hay más caracteres en la cinta de entrada, pasa a qf.

q1 (Estado restante): Aplica la operación módulo 26 a la representación unaria en la cinta temporal. Pasa a q2.

q2 (Estado convertir): Convierte la representación unaria resultante (después del módulo) de la cinta temporal de nuevo a un carácter del alfabeto. Escribe este carácter cifrado en la cinta de salida. Pasa a q3.

q3 (Estado desplazar): Mueve los cabezales de las cintas de entrada, clave y salida una posición a la derecha. Reinicia la cinta temporal y su cabezal. Pasa de nuevo a q0 para procesar el siguiente carácter.

qf (Estado final):Estado de aceptación. Indica que el proceso de cifrado ha terminado para toda la cadena de entrada.

Σ (Alfabeto de entrada): El alfabeto de los símbolos de entrada permitidos para el mensaje y la clave. Definido al principio del codigo

Γ (Alfabeto de cinta): El conjunto de símbolos que pueden ser escritos o leídos en las cintas de la MT.

Γ = Σ ∪ {1, _}

Donde 1 se utiliza para la representación unaria de los valores numéricos de los caracteres, y '_' representa el símbolo blanco o espacio vacío en las cintas.

q0 (Estado inicial): El estado inicial de la MT es q0 (representado internamente como 'inicio').

B (Símbolo blanco): El símbolo blanco utilizado en las cintas es _. Se usa para inicializar la cinta de salida y para representar el contenido vacío de la cinta temporal al inicio de cada ciclo de cifrado de un carácter.

F = {qf} (representado como 'fin')

### Descripción de la Función de Transición (δ)

Como fue explicado en la tarea,: dada la potencial complejidad de presentarla explícitamente debido el gran número de transiciones posibles para una tarea como esta, describimos la lógica general de la función de transición δ, la cual está implementada en el método paso() de la clase MaquinaDeTuring. La MT opera de la siguiente manera según su estado actual y los símbolos leídos:

Desde el estado q0:

Condición: El puntero de la cinta de entrada no ha alcanzado el final del texto.

Lectura: Lee el carácter del mensaje de la cinta de entrada en la posición actual del puntero y el carácter de la clave en la posición actual del puntero.

Operación Interna (simulada):

Convierte el caracter del mensaje y de la clave a su representación unaria

Escritura: Escribe la concatenación de los dos resultados anteriores en la cinta temporal. El cabezal de la cinta temporal se posiciona al inicio.

Movimiento: Los cabezales de las cintas de entrada, clave y salida no se mueven en esta transición específica.

Nuevo Estado: Pasa al estado q1.

Condición Alternativa: Si el puntero de la cinta de entrada ha alcanzado el final del texto, la maquina transita directamente al estado qf y la máquina se detiene.

Desde el estado q1:

Lectura: No lee explícitamente de las cintas de entrada/clave/salida para la transición, trabaja sobre la cinta temporal.

Operación Interna (simulada): Aplica la operación módulo 26 a la cadena de 1s que esta en la cinta temporal. Esto se simula reduciendo la longitud de la cadena de 1s a longitud % 26.

Escritura: La cinta temporal se actualiza con el resultado de la operación módulo.

Movimiento: No hay movimiento de cabezales de las cintas principales.

Nuevo Estado: Pasa al estado q2.

Desde el estado q2:

Lectura: Opera sobre la cinta temporal.

Operación Interna (simulada): Convierte la representación unaria de la cinta temporal de nuevo a un carácter del alfabeto (ej."1" → B, ...). Siendo este el carácter cifrado.

Escritura: Escribe el carácter cifrado en la cinta de salida, en la posición actual del puntero de la cinta de salida.

Movimiento: No hay movimiento de cabezales de las cintas principales.

Nuevo Estado: Transita al estado q3.

Desde el estado q3:

Lectura: No aplica.

Operación Interna: Prepara la maquina para el siguiente carácter.

Escritura: La cinta temporal se limpia (se vacía o se llena de símbolos blancos _) y su puntero se resetea a la posición 0.

Movimiento: Los punteros de las cintas de entrada, clave y salida se mueven una posición a la derecha (D).

Nuevo Estado: Transita de nuevo al estado q0 para procesar el siguiente par de caracteres.

Estado qf:

Este es un estado de parada. La máquina no realiza más transiciones. El contenido de la cinta de salida representa el texto cifrado completo.

La lógica de conversión a/desde unario y la operación módulo, aunque se describen como operaciones internas de la MT, son implementadas en la simulación mediante la clase auxiliar ProcesadorUnario para mayor eficiencia y claridad del código de la MT. Una MT "pura" realizaría estas operaciones mediante una secuencia más detallada de estados y movimientos de cabezal para manipular los '1's en la cinta temporal.

hemos optado por utilizar estructuras condicionales (if/elif/else) dentro del método paso() de la clase MaquinaDeTuring. Consideramos que este enfoque facilita la lógica de las transiciones que comparten patrones similares y mejora la legibilidad general del código, especialmente dado el flujo secuencial de operaciones dentro de cada ciclo de cifrado de un carácter.

### Configuración de Cintas

Para implementar el cifrado Vigenère, hemos optado por una configuración de cuatro cintas. Consideramos que esta configuración ofrece un buen equilibrio entre claridad conceptual y eficiencia para simular las operaciones requeridas. La justificación y el propósito de cada cinta son los siguientes:

Cinta 1: Cinta de Entrada

Propósito: Almacena el mensaje en claro que se va a cifrar.

Funcionamiento: Es una cinta de solo lectura en términos de su contenido original. El cabezal se mueve de izquierda a derecha, leyendo un carácter a la vez.

Justificación: Necesaria para acceder secuencialmente a cada carácter del mensaje original.

Cinta 2: Cinta de Clave

Propósito: Almacena la palabra clave, repetida o extendida para que coincida con la longitud del mensaje en claro.

Funcionamiento: Similar a la cinta de entrada, es de solo lectura para su contenido original. El cabezal se mueve de izquierda a derecha en sincronía con el cabezal de la cinta de entrada, leyendo el carácter correspondiente de la clave.

Justificación: Permite aplicar el desplazamiento correcto del cifrado Vigenère para cada carácter del mensaje, según el carácter correspondiente de la clave.

Cinta 3: Cinta de Salida

Propósito: Almacena el mensaje cifrado final.

Funcionamiento: Inicialmente está llena de símbolos blancos (_). A medida que cada carácter del mensaje en claro se cifra, el carácter resultante se escribe en la posición correspondiente de esta cinta. El cabezal se mueve de izquierda a derecha.

Justificación: Necesaria para construir y almacenar el resultado final del proceso de cifrado.

Cinta 4: Cinta Temporal o de Cálculo

Propósito: Utilizada como espacio de trabajo para realizar las operaciones aritméticas fundamentales del cifrado: la conversión de caracteres a su representación unaria, la suma de estos valores y la aplicación del módulo 26.

Funcionamiento: Esta cinta es de lectura y escritura.

En el estado q0, se escriben en ella las representaciones unarias concatenadas del carácter del mensaje y el carácter de la clave.

En el estado q1, se manipula su contenido para aplicar el módulo 26.

En el estado q2, se lee su contenido para convertirlo de nuevo a un carácter.

En el estado q3, se limpia o reinicia para el siguiente ciclo de cifrado.

Justificación: Abstrae las operaciones aritméticas que una MT real realizaría mediante la manipulación de símbolos. El uso de una cinta dedicada para estos cálculos mantiene las otras cintas (entrada, clave, salida) limpias y enfocadas en sus roles primarios, simplificando la lógica general de la máquina. Aunque la simulación en Python usa una clase auxiliar para estas operaciones, esta cinta representa el espacio donde una MT "física" realizaría dichas manipulaciones.

Esta configuración de cuatro cintas permite una separación clara de las funciones y facilita la simulación del proceso de cifrado Vigenère de una manera organizada y comprensible.